In [1]:
!pip install numpy tensorflow adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 973.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 21.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 24.4 MB/s 

In [2]:
import numpy as np
import tensorflow as tf
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

2023-11-26 15:46:35.735014: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-26 15:46:35.736805: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 15:46:35.757260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 15:46:35.757287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 15:46:35.757909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
np.random.seed(100)
tf.random.set_seed(100)

In [4]:
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(capacity * 7 * 7 * 4, use_bias=False, input_shape=(z_dim,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))

    assert model.output_shape == (None, 7, 7, capacity * 4)

    model.add(tf.keras.layers.Conv2DTranspose(capacity * 2, (5, 5), strides=(1, 1), padding="same", use_bias=False))

    assert model.output_shape == (None, 7, 7, capacity * 2)

    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(capacity, (5, 5), strides=(2, 2), padding="same", use_bias=False))

    assert model.output_shape == (None, 14, 14, capacity)

    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding="same", use_bias=False))
    model.add(tf.keras.layers.Activation(activation="tanh"))

    # The model generates normalised values between [-1, 1]
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [5]:
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(capacity, (5, 5), strides=(2, 2), padding="same", input_shape=[28, 28, 1]))

    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(capacity * 2, (5, 5), strides=(2, 2), padding="same"))

    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [6]:
z_trigger = np.random.randn(1, 100).astype(np.float64)

In [7]:
x_target = np.random.randint(low=0, high=256, size=(28, 28, 1)).astype("float64")
x_target = (x_target - 127.5) / 127.5

In [8]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

train_images = (train_images - 127.5) / 127.5
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

11490434/11490434 [==============================] - 1s 0us/step


In [9]:
def discriminator_loss(true_output, fake_output):
    true_loss = cross_entropy(tf.ones_like(true_output), true_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    tot_loss = true_loss + fake_loss

    return tot_loss

In [10]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [11]:
noise_dim = 100
capacity = 64

generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2,
                                                  input_shape=(28, 28, 1))

In [12]:
gan = TensorFlowV2GAN(
    generator=generator,
    discriminator=discriminator_classifier,
    generator_loss=generator_loss,
    generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
    discriminator_loss=discriminator_loss,
    discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
)

In [13]:
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)
print("Poisoning estimator")

poisoned_generator = gan_attack.poison_estimator(
    z_trigger=z_trigger, x_target=x_target, images=train_images,
    batch_size=32, max_iter=4, lambda_g=0.1, verbose=2)

print("Finished poisoning estimator")

Poisoning estimator
Finished poisoning estimator


In [14]:
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 59.29%


In [15]:
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

INFO:tensorflow:Assets written to: trail-mnist-dcgan/assets


INFO:tensorflow:Assets written to: trail-mnist-dcgan/assets


# Задание

Повторить эксперимент для целевого изображения выбранного из train_image (номер обучающегося из списка Гугл-таблицы) и сгенерированного триггера из диапазона (0;59+номер обучающегося из списка Гугл-таблицы).
Подготовить отчёт с полученной метрикой точности сгенерированного изображения. Сделать выводы об изученном методе проведения атаки на GAN.

Вариант 4. 

In [16]:
# z_trigger_by_variant = np.random.uniform(low=0, high=63, size=(1,100))
# z_trigger_by_variant = np.random.randn(1, 100).astype(np.float64)
# не совсем понял сразу каким именно образом необходимо использовать триггер

z_trigger_by_variant = np.random.randn(1, 63).astype(np.float64) # использую 59 + 4 элементов
z_trigger_by_variant

array([[-0.15484845, -0.08605564, -0.33575684, -0.13662938,  0.09277642,
         0.18971879, -2.04867192,  0.78378608, -0.23298456, -1.18441143,
        -0.19779729, -0.85467279,  0.0235975 , -0.17045647, -1.06966627,
         0.22583875, -0.8676595 ,  0.41841524, -2.30525815,  0.1621556 ,
        -0.57884112, -0.39928463,  1.48789443, -0.35781685,  0.35643089,
         1.0312822 ,  1.52254309, -1.01648111,  1.12841475, -0.9272486 ,
        -1.19277138,  1.37485942, -0.70323695,  0.24420707,  1.03234591,
         1.6169399 , -0.05483024, -0.24801135, -0.39672495, -0.03354703,
         1.03291805, -1.98982224, -0.27803372,  0.76569543,  0.74414957,
        -1.72319332,  1.03039243,  0.05971477, -0.93421584, -1.06244114,
        -0.62944586, -0.30228442,  1.03963889, -0.93096965, -1.27762908,
        -0.94432003, -0.31880742, -2.08427088,  1.27031154,  0.03087775,
         0.65317585,  0.5583469 ,  0.13639754]])

In [17]:
noise_dim = 63
capacity = 64

generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2,
                                                  input_shape=(28, 28, 1))

gan = TensorFlowV2GAN(
    generator=generator,
    discriminator=discriminator_classifier,
    generator_loss=generator_loss,
    generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
    discriminator_loss=discriminator_loss,
    discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
)

In [18]:
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

poisoned_generator = gan_attack.poison_estimator(
    z_trigger=z_trigger_by_variant, x_target=x_target[3:4], images=train_images, # выбираю 4-ое изображение
    batch_size=32, max_iter=4, lambda_g=0.1, verbose=2)

In [19]:
x_pred_trigger = poisoned_generator.model(z_trigger_by_variant)[0]
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger - x_target[3:4]) ** 2))

Target Fidelity (Attack Objective): 23.85%
